In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("mylogistic").getOrCreate()

In [3]:
my_data = spark.read.csv('/FileStore/tables/customer_churn.csv', header = True, inferSchema = True)

In [4]:
my_data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [5]:
my_data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [6]:
my_data.columns

Out[ 8 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                           outputCol = 'features')

In [9]:
output = assembler.transform(my_data)

In [10]:
final_data = output.select('features', 'churn')

In [11]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr_churn = LogisticRegression(labelCol = 'churn')

In [14]:
fit_lr_churn = lr_churn.fit(train_data)

In [15]:
training_sum = fit_lr_churn.summary

In [16]:
training_sum.predictions.describe().show()

+-------+------------------+------------------+
summary| churn| prediction|
+-------+------------------+------------------+
 count| 625| 625|
 mean| 0.176| 0.1392|
 stddev|0.3811251897427014|0.3464323665816817|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+------------------+

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fit_lr_churn.evaluate(test_data)

In [19]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[4.09928579820402...|[0.98368604321113...| 0.0|
[25.0,9672.03,0.0...| 0|[4.52235201191958...|[0.98925330347110...| 0.0|
[26.0,8939.61,0.0...| 0|[6.26122446290194...|[0.99809473000394...| 0.0|
[28.0,8670.98,0.0...| 0|[7.74569670774269...|[0.99956758691630...| 0.0|
[28.0,9090.43,1.0...| 0|[0.94999056772918...|[0.72111328111305...| 0.0|
[28.0,11128.95,1....| 0|[3.78011792058870...|[0.97768913365846...| 0.0|
[28.0,11204.23,0....| 0|[1.47237337786076...|[0.81341785977082...| 0.0|
[28.0,11245.38,0....| 0|[3.74005587586095...|[0.97679832822252...| 0.0|
[29.0,12711.15,0....| 0|[5.44931236293380...|[0.99571914450573...| 0.0|
[30.0,8403.78,1.0...| 0|[5.54030882860568...|[0.99609003344629...| 0.0|
[30.0,8874.83,0.0...| 0|[3.02591737486179...|[0.95373134879821...| 0.0|
[30.0,10183.98,1....| 0|[2.43220371569725...|[0.91925026407963...| 0.0|
[30.0,10960.52,1....| 0|[1.99616533876998...|[0.88039387487223...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.19736566968146...|[0.96073502225625...| 0.0|
[31.0,8829.83,1.0...| 0|[4.01471704051366...|[0.98227189717752...| 0.0|
[31.0,9574.89,0.0...| 0|[3.33669437995697...|[0.96566641371034...| 0.0|
[31.0,10058.87,1....| 0|[4.23165779364269...|[0.98567976282277...| 0.0|
[32.0,7896.65,0.0...| 0|[3.44380340182962...|[0.96904580655425...| 0.0|
[32.0,10716.75,0....| 0|[4.41083500205479...|[0.98800069893964...| 0.0|
[32.0,11540.86,0....| 0|[6.75728810343974...|[0.99883897240187...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [20]:
eval_churn = BinaryClassificationEvaluator(labelCol= 'churn',
                                          rawPredictionCol = 'prediction')

In [21]:
auc = eval_churn.evaluate(pred_and_labels.predictions)
auc

Out[ 31 ]: 0.8140957446808511

In [22]:
#predict on new data
final_lr_model = lr_churn.fit(final_data)

In [23]:
new_data = spark.read.csv('/FileStore/tables/new_customers.csv', header = True,
                         inferSchema = True)

In [24]:
test_new_data = assembler.transform(new_data)

In [25]:
new_results = final_lr_model.transform(test_new_data)

In [26]:
new_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.7691621189402...|[0.02255110110413...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-5.0956222016519...|[0.00608622642084...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.10475867224153...|[0.75115067517475...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.6896019277064...|[0.15582819767635...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+

In [27]:
new_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+